In [4]:
import json
import pandas as pd
import random
import mp
import os
from glob import glob
from tqdm import tqdm
from transformers import AutoProcessor

In [5]:
processor = AutoProcessor.from_pretrained("Qwen/Qwen2-Audio-7B-Instruct")
tokenizer = processor.tokenizer
audio_token = "<|AUDIO|>"
audio_bos_token = "<|audio_bos|>"
audio_eos_token = "<|audio_eos|>"
audio_token_id = processor.tokenizer._convert_token_to_id_with_added_voc('<|AUDIO|>')
pad_token_id = processor.tokenizer.pad_token_id

In [2]:
data = pd.read_parquet('/home/husein/ssd3/dataset/speech-instructions/speech-instructions.parquet').to_dict(orient = 'records')
data[0]

{'prompt': '[{"role": "user", "content": [{"type": "audio", "audio_url": "audio.wav"}]}, {"role": "assistant", "content": "Untuk mengendalikan ralat pada laman web menggunakan JavaScript, anda boleh mengikuti langkah-langkah berikut:\\n\\n1. Gunakan try...catch: Salah satu cara untuk menangani ralat dalam JavaScript adalah dengan menggunakan blok try...catch. Blok try digunakan untuk menguji sebahagian kod yang mungkin menghasilkan ralat, manakala blok catch digunakan untuk menangkap dan memproses ralat yang ditangkap.\\n\\nContoh:\\n```javascript\\ntry {\\n  // Kod yang mungkin menghasilkan ralat\\n} catch (error) {\\n  // Tangani ralat di sini\\n}\\n```\\n\\n2. Gunakan onerror: anda juga boleh menggunakan acara onerror untuk menangani ralat yang berlaku semasa pemuatan dokumen atau skrip. Acara ini akan dipicu apabila terdapat ralat yang tidak ditangani.\\n\\nContoh:\\n```javascript\\nwindow.onerror = function(errorMessage, fileName, lineNumber) {\\n  // Tangani ralat di sini\\n};\\n

In [14]:
def loop(rows):
    rows, _ = rows
    data = []
    for r in tqdm(rows):
        try:
            conversation = json.loads(r['prompt'])
            text = processor.apply_chat_template(conversation, tokenize=False)
            f = r['audio_filename']
            if 'dialects_processed' in f:
                f = os.path.join('/home/husein/ssd3', f)
            else:
                f = os.path.join('/home/husein/ssd3/dataset/speech-instructions', f)

            data.append({
                'text': text,
                'audio': f,
            })
        except:
            pass
    return data

In [15]:
processed = loop((data[:100], 0))

100%|███████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 7129.65it/s]


In [16]:
processed[0]

{'text': '<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n<|im_start|>user\nAudio 1: <|audio_bos|><|AUDIO|><|audio_eos|>\n<|im_end|>\n<|im_start|>assistant\nUntuk mengendalikan ralat pada laman web menggunakan JavaScript, anda boleh mengikuti langkah-langkah berikut:\n\n1. Gunakan try...catch: Salah satu cara untuk menangani ralat dalam JavaScript adalah dengan menggunakan blok try...catch. Blok try digunakan untuk menguji sebahagian kod yang mungkin menghasilkan ralat, manakala blok catch digunakan untuk menangkap dan memproses ralat yang ditangkap.\n\nContoh:\n```javascript\ntry {\n  // Kod yang mungkin menghasilkan ralat\n} catch (error) {\n  // Tangani ralat di sini\n}\n```\n\n2. Gunakan onerror: anda juga boleh menggunakan acara onerror untuk menangani ralat yang berlaku semasa pemuatan dokumen atau skrip. Acara ini akan dipicu apabila terdapat ralat yang tidak ditangani.\n\nContoh:\n```javascript\nwindow.onerror = function(errorMessage, fileName, lineNumber) {\n  // T

In [17]:
processed = mp.multiprocessing(data, loop, cores = 10)

100%|███████████████████████████████████████████████████████████████████████████████| 46947/46947 [00:06<00:00, 6868.61it/s]


In [18]:
len(processed)

467010

In [21]:
processed[1000]

{'text': '<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n<|im_start|>user\nAudio 1: <|audio_bos|><|AUDIO|><|audio_eos|>\n<|im_end|>\n<|im_start|>assistant\nDalam Next.js, anda boleh menambahkan pengendalian ralat pada permintaan HTTP dengan menggunakan `try...catch` atau menggunakan perpustakaan seperti `axios` yang mempunyai fungsi pengendalian ralat bawaan. Berikut adalah contoh bagaimana anda boleh melakukannya:\n\n### Menggunakan try...catch\n\n```javascript\nimport { useRouter } from \'next/router\';\n\nconst router = useRouter();\n\nasync function fetchData() {\n  try {\n    const response = await fetch(\'https://api.example.com/data\');\n    const data = await response.json();\n    // Lakukan sesuatu dengan data\n  } catch (error) {\n    console.error("Terjadi kesalahan:", error);\n    router.push(\'/error-page\'); // Alih ke halaman ralat\n  }\n}\n\nexport default function MyComponent() {\n  fetchData();\n  return (\n    <div>\n      {/* Kandungan komponen */}\n   

In [22]:
with open('prepare-speech-instructions.json', 'w') as fopen:
    json.dump(processed, fopen)